# Malek et al. (2015) Model Code <a name="top" />

# Table of Contents
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
3. [Parameters and Initial Conditions](#params)
4. [Put Raw Data into Arrays](#rawdata)
    1. [Plot Data Sets](#plotdata)
5. [Model Function--Includes ODE Solver](#modelfunction)
6. [Functions to Determine Delayed ACTH and Cortisol Values](#delays)
7. [Delay Functions for Larger Time Steps](#largerdelays)
8. [Cost Function Definition](#cost)
9. [Run the Optimization](#run)
10. [Save Output to File](#saveoutput)
11. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
12. [Plots](#plots)
13. [No Optimization Run](#no-opt)
14. [Run Optimizations for Multiple Patients](#runMultiple)
15. [Dependencies](#dependencies)

# Instructions <a name="instructions"></a>

## Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the heading **Run the Optimization**. Simply run all cells up to the cell below that heading.

In order to set which data set to optimize parameters against, look for the following line of code:
    
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

In order to run against a patient from the TSST data sets, simply change the list entries to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to set as the first (ACTH time steps) and third (cortisol time steps) indices of the `data_to_match` list.

The following are several examples of lists you could use for parameter optimization with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1]`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14]`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2]`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1]`
    - The mean of all control patients
    
It is unlikely that you will need to optimize any initial conditions (ICs), because the only ICs other than ACTH and cortisol are inflammatory cytokines. However, if necessary, this can be modified in the function `cost_fun(params)`. In the example below, we use the first three optimized parameters in the list returned by the optimization algorithm to set the ICs we want to optimize (LPS, TNF-alpha and IL-6 in this example):

    y0 = [params[0], params[1], params[2], y0[1], y0[2]]

We then need to pass only the remaining parameters in the list to the model, along with the updated ICs in y0:

    simData = model(params[3:], y0)
    
If you want to not optimize any ICs, you would simply leave the cost function as it is currently written.

At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

**Note:** You also have the option of using a cost function based on the maximum distance between simulation and real-world data. Simply change SSE_cost to MAX_cost, the instructions for function arguments remain the same.

The cell directly under the heading **Save Output to File** can be changed so that the root filename matches the simulations being run. This root will be used to save all of the various data and figures generated. The current naming scheme would save the files for 5 iterations of parameter optimization against the mean data set from the Nelson with ICs for CRH and GR optimized as:

    filename_root = "malekModel_output/nelson-patientMean-5-iterations-ICOpt"

This saves the files in a subfolder specific to this model, which helps keep files organized when running multiple models.

The next few cells create an Excel file containing all of the concentration data and optimized parameter values, and text files containing the initial conditions, parameter bounds and parameter means +- standard deviation across the 5 iterations.

The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** creates an instance of the Visualizer object from the VeVaPy module called visualize. This will start a dialog which asks for several inputs to generate figures as desired.

After initialization of the object, run its method `make_graphs()`, and it will generate figures using the data you have specified. There are a number of arguments that can be optionally specified for this method, and you can see more details by running the command:

    help(Visualizer)

    
## Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period (1440 minutes), rather than 140.5 minutes, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and uncomment (delete the # at the start of the line) the lines:

    t_start = -0.5
    t_end = 1455.5
    t_step = 0.5

You'll need to comment out the other definitons for these variables (place a # at the start of the line).

The reason you add the extra 15 minutes is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 1455 minutes).

After making this change, you need to again change the `data_to_match` list so that you are matching the basal data set in which you are interested.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). Just as with the Nelson data, in all of these data sets the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to use as the first (ACTH time steps) and third (cortisol time steps) indices in the `data_to_match` list.

Then for the second and fourth indices, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing lists you can use for optimization:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1]`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1]`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the cost function to `optimize.SSE_cost_noACTH` and then update `data_to_match` to not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]
    return optimize.SSE_cost_noACTH(data_to_match[0], data_to_match[1], simData)

For data without ACTH concentrations, you will also need to comment out the current definition of `y0` and uncomment the following line (and change the IC to the desired value for ACTH):

    #y0 = [0, 0, 0, 10, data_to_match[1][0]]
    
At this point, you're ready to run the parameter optimization, so run all of the cells under the heading **Run the Optimization**.

The cell directly under the heading **Save Output to File** should again have the filename changed to something that reflects the data set you're matching now. For instance, the filename root when matching the smoothed Carroll Control group and optimizing ICs would become:

    filename_root = 'malekModel_output/carroll-control-smooth-5-iterations-ICopt'
            
Finally, the cells under the heading **Plots** should be run again to generate graphs. The same process of giving inputs to the object dialog will be performed and then the method `make_graphs()` should be run with any optional arguments desired.

## Running Without Parameter Optimization <a name="noOptInstructions" />

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run**. Change the parameters and initial conditions defined under the heading **Parameters and Initial Conditions**, and then run the cell containing the following line:

    data_no_opt = model(authors_params, y0)
    
Then run the cells under **Plots** to create graphs as described in the sections regarding simulations with parameter optimization.

[Back to Top](#top)

# Imports <a name="imports"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
from scipy import optimize
from scipy.interpolate import interp1d
import mpld3
from tabulate import tabulate
import pandas as pd
from VeVaPy import DEsolver, optimize
from VeVaPy.dataImport import data
from VeVaPy.visualize import Visualizer

[Back to Top](#top)

# Parameters and Initial Conditions <a name="params"></a>

In [ ]:
# initial conditions
# order is: LPS endotoxin, TNF-alpha, IL-6, ACTH, Cortisol

y0 = [0, 0, 0,14.47489,1.903323]

In [ ]:
# authors' published parameters

e_P = 0.05
e_T = 0.038
e_S = 0.02
e_A = 0.04
e_C = 0.01
d_1 = 0.026
d_2 = 0.068
d_3 = 0.063
d_4 = 2.37
d_5 = 9.39
d_6 = 0.35
k = 0.0504
tau_1 = 10
tau_2 = 10
m_1 = 4
m_2 = 4
c = 6.11
a = 21
h = 7.66
alpha = 0.28
x_1 = 3.25
x_2 = 0.86
x_3 = 0.016
x_4 = 6.11
x_5 = 1.39
x_6 = 1.57
x_7 = 6.11
x_8 = 1.72
x_9 = 0.87
x_10 = 0.94
x_11 = 1.87
x_12 = 1.97

authors_params = [e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12]

In [ ]:
# compute bounds based on +- 10%
bound = x_7
print(bound - bound*.1)
print(bound + bound*.1)

In [ ]:
# bounds for parameter optimization
# starting with +- 10% since we do not have published ranges in the paper
# order is: e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12
# the authors included 95% CI for several of the parameters, so we have used those intervals as the bounds for
#     the parametrs they included
# the ones included are: x_1, x_2, x_3, x_5, x_6, x_8, x_9, x_10, x_11, x_12, k, d_1, d_2, d_3, d_4, d_5, d_6
bounds = [(0.045, 0.055), (0.0342, 0.0418), (0.018, 0.022), (0.036, 0.044), (0.009, 0.011), (0.021, 2.21), (0.052, 1.39), (0.0074, 0.3), (0.71, 2.63), (8.59, 13.55), (0.27, 2.46), (0.03, 0.54), (9., 11.), (9., 11.), (3.6, 4.4), (3.6, 4.4), (5.499, 6.721), (18.9, 23.1), (6.894, 8.426), (0.252, 0.308), (2.62, 6.13), (0.09, 2.02), (0.013, 3.4), (5.499, 6.721), (0.81, 3.79), (2.81, 5.68), (4.499, 6.721), (0.99, 5.16), (0.27, 4.07), (0.05, 2.6), (0.2, 5.43), (1.47, 6.45)]


In [ ]:
# define time interval for integration

# time interval and step definition
# all data sets end on 1440.0 or earlier except the Golier cortisol sets,
# they end on 1455.0, so I should set t_end = 1455.01 when matching them
#t_start = -0.01
#t_end = 1455.01
#t_step = 0.01

# for matching Nelson data, use these values of t_start, t_end and t_step
t_start = -0.5
t_end = 140.5
t_step = 0.5

[Back to Top](#top)

# Put Raw Data into Arrays <a name="rawdata"></a>

In [ ]:
# Create an instance of the data class for each data set contained in the VeVaPy library, and set the time
# scale to hours.
yehuda = data("yehuda", "minutes")
carroll = data("carroll", "minutes")
golier = data("golier", "minutes")
bremner = data("bremner", "minutes")
nelson = data("nelson", "minutes")

## Plot Data Sets <a name="plotdata"></a>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
#mpld3.enable_notebook()
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (15,15))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15, 15))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,1])
ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,1])

[Back to Top](#top)

# Model Function -- Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    # function containing the system, to be called by solver
    def ode_system(t, y):
        # initialize array to hold ODE function values
        dy = np.zeros(5)
        #dy = np.zeros(2)
        
        # define parameter values
        [e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12] = params

        # the ODE system itself
        dy[0] = -e_P*y[0] - d_1*(y[1]/(x_1 + y[1]))*(y[2]/(x_2 + y[2]))*y[0]
        dy[1] = -e_T*y[1] + k*(y[0]/(x_3 + y[0])) + ((d_2/(x_4 + y[4])) + (d_3/(x_5 + y[2])))*(y[1]/(x_6 + y[1]))
        dy[2] = -e_S*y[2] + d_4*(1/(x_7 + y[4]))*(y[1]/(x_8 + y[1]))
        # to include circadian input function, uncomment line with E(t) included in ACTH
        #dy[3] = -e_A*y[3] + E(t)*h*((c**m_1)/(c**m_1 + DEsolver.delayedCORT**m_1)) + d_5*(y[2]/(x_9 + y[2]))*(y[1]/(x_10 + y[1]))
        dy[3] = -e_A*y[3] + h*((c**m_1)/(c**m_1 + DEsolver.delayedCORT**m_1)) + d_5*(y[2]/(x_9 + y[2]))*(y[1]/(x_10 + y[1]))
        dy[4] = -e_C*y[4] + alpha*((DEsolver.delayedACTH**m_2)/(DEsolver.delayedACTH**m_2 + a**m_2)) + d_6*(y[2]/(x_11 + y[2]))*(y[1]/(x_12 + y[1]))
        
        return dy
    
    def E(t):
        return (0.6 + 0.171*np.sin(((2*np.pi)/(24*60))*t) - 0.4698*np.cos(((2*np.pi)/(24*60))*t))
    
    # Call the solve() function from my DEsolver module, and pass all of the information it needs.
    # Arguments are as follows: ODE function to solve, array of initial conditions, start time, step size, end time, 
    # The last five arguments are optional (leave blank for ODE systems) for delay differential equation systems,
    #  tau1 is the delay in ACTH, tau2 is the delay in CORT, and delay is an array of booleans to set whether 
    #  we use delays in [CRH, ACTH, CORT]
    #
    # Because of the fact that this model has equations for LPS, TNF-alpha and IL-6 before the equations for ACTH & Cortisol,
    #  we need to change the optional arguments y_index1 to 3 (for the index of ACTH in the array y), and y_index2 to 4 (for the 
    #  index of cortisol in the array y). 
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end, y_index1 = 3, y_index2 = 4, tau1 = 10, tau2 = 10, delay = [False, True, True], delay_rough = True)
    return timeSeries

[Back to Top](#top)

# Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    global y0, data_to_match
    simData = model(params, y0)
    
    # To optimize ICs for the inflammatory cytokines, uncomment below and comment the line starting with simData above
    #y0 = [params[0], params[1], params[2], y0[3], y0[4]]
    #simData = model(params[3:], y0)
    
    # To optimize the IC for ACTH (and not the inflammatory cytokines), in the case that you are matching basal data
    #  with only cortisol concentrations, uncomment the following lines
    #y0 = [0, 0, 0, params[0], y0[4]]
    #simData = model(params[1:], y0)
    
    return optimize.SSE_cost(data_to_match[0], data_to_match[1], data_to_match[2], data_to_match[3], simData)

[Back to Top](#top)

# Run the Optimization <a name="run"></a>

In [ ]:
# Define the data set to match with the parameter optimization algorithm.
# Requires 4 indices, in the order:
# ACTH time steps, ACTH data, Cortisol time steps, Cortisol data
data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

# For matching data with only cortisol concentrations, use the following line and change the data sets as desired:
#data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]

In [ ]:
# initial conditions
# order: LPS endotoxin, TNF-alpha, IL-6, ACTH, CORT

# Setting the inflammation-related ICs to zero turns the model into one of just the HPA axis
y0 = [0, 0, 0, data_to_match[1][0], data_to_match[3][0]]

# For matching data with only cortisol concentrations, use the following line and change the ICs as desired:
#y0 = [0, 0, 0, 10, data_to_match[1][0]]

In [ ]:
# We call the run() method of the optimize module, which will run the parameter optimization given the arguments we pass
# Required arguments are the cost function, the model, and the real-world data we want to match
#
# Optional arguments include the initial conditions we want to optimize (if any), the number of iterations to run,
#  the maximum number of optimization steps per iteration of the algorithm, the algorithm to use (defaults to 
#  differential_evolution), and the popsize to use (larger popsize gives more accurate optimization but is more 
#  computationally expensive) 
opt_pars, simData_all = optimize.run(cost_fun, model, data_to_match, y0, bounds, num_iter=5)


[Back to Top](#top)

# Save Output to File <a name="saveoutput"></a>

In [ ]:
# Change the root filename, this will have the array name appended to it
#  to make the filename of the Excel files
filename_root = "malekModel_output/nelson-patientMean-5-iterations-ICOpt"

In [ ]:
# Create the pandas DataFrame object for opt_pars
# I've typed out each individual heading for the parameter names that were
#  optimized, and assigned the correct column of opt_pars to them
#
# NOTE: I've been unable to get this code to work for 1 iteration of parameter optimization (only for opt_pars)
# Hopefully in the near future I'll get it worked out, but for some reason I can't get it to be output as a single row
# with 21 columns. It'll only output as a single column with 21 rows.
df_opt_pars = pd.DataFrame(opt_pars, columns=['Cost',
                                              'e_P',
                                              'e_T',
                                              'e_S',
                                              'e_A',
                                              'e_C',
                                              'd_1',
                                              'd_2',
                                              'd_3',
                                              'd_4',
                                              'd_5',
                                              'd_6',
                                              'k',
                                              'tau_1',
                                              'tau_2',
                                              'm_1',
                                              'm_2',
                                              'c',
                                              'a',
                                              'h',
                                              'alpha',
                                              'x_1',
                                              'x_2',
                                              'x_3',
                                              'x_4',
                                              'x_5',
                                              'x_6',
                                              'x_7',
                                              'x_8',
                                              'x_9',
                                              'x_10',
                                              'x_11',
                                              'x_12'])
# Create the pandas DataFrame object for simData_all
# I've typed out each individual heading for the variables and iterations,
# and assigned the correct column of simData_all to them
df_simData_all = pd.DataFrame(simData_all, columns=['Iteration 1 Time',
                                                    'Iteration 1 ACTH',
                                                    'Iteration 1 Cortisol',
                                                    'Iteration 1 LPS',
                                                    'Iteration 1 TNF-alpha',
                                                    'Iteration 1 IL-6',
                                                    'Iteration 2 Time',
                                                    'Iteration 2 ACTH',
                                                    'Iteration 2 Cortisol',
                                                    'Iteration 2 LPS',
                                                    'Iteration 2 TNF-alpha',
                                                    'Iteration 2 IL-6',
                                                    'Iteration 3 Time',
                                                    'Iteration 3 ACTH',
                                                    'Iteration 3 Cortisol',
                                                    'Iteration 3 LPS',
                                                    'Iteration 3 TNF-alpha',
                                                    'Iteration 3 IL-6',
                                                    'Iteration 4 Time',
                                                    'Iteration 4 ACTH',
                                                    'Iteration 4 Cortisol',
                                                    'Iteration 4 LPS',
                                                    'Iteration 4 TNF-alpha',
                                                    'Iteration 4 IL-6',
                                                    'Iteration 5 Time',
                                                    'Iteration 5 ACTH',
                                                    'Iteration 5 Cortisol',
                                                    'Iteration 5 LPS',
                                                    'Iteration 5 TNF-alpha',
                                                    'Iteration 5 IL-6'])

# Create an instance of the ExcelWriter class and open the file using a with statement
with pd.ExcelWriter(filename_root+".xlsx") as writer:
    # Define the header format, so that it's bold, text is wrapped, it has a 
    #  colored background and a border
    header_format = writer.book.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    # Write the opt_pars array to a sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also change the row index to start at 1, rather than 0.
    df_opt_pars.index = list(range(1,len(opt_pars[:,0])+1))
    df_opt_pars.to_excel(writer, sheet_name="Optimized Parameters", startrow=1, header=False)
    
    # Write the simData_all array to another sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also disable the row index numbers, as they are not necessary here.
    df_simData_all.to_excel(writer, sheet_name="Simulated Concentration Data", startrow=1, header=False, index=False)
    
    # Loop through each header in opt_pars DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_opt_pars.columns.values):
        # We write in the sheet "Optimized Parameters" in the first row, starting with the second column 
        #  (because of the row indices), using the headers from the DataFrame and the header format we defined above
        writer.sheets["Optimized Parameters"].write(0, col+1, val, header_format)
    
    # Loop through each header in simData_all DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_simData_all.columns.values):
        # We write in the sheet "Simulated Concentration Data" in the first row, starting with the first column 
        #  (because we turned off the row indices), using the headers from the DataFrame and 
        #  the header format we defined above
        writer.sheets["Simulated Concentration Data"].write(0, col, val, header_format)
    

In [ ]:
# Save the initial conditions and bounds to text files, also.
np.savetxt(filename_root+'-initial-conditions.txt', y0)
np.savetxt(filename_root+'-bounds.txt', bounds)

# Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
# compute parameter means and standard deviations
e_P_mean = np.mean(opt_pars[:,1])
e_P_std = np.std(opt_pars[:,1])
e_T_mean = np.mean(opt_pars[:,2])
e_T_std = np.std(opt_pars[:,2])
e_S_mean = np.mean(opt_pars[:,3])
e_S_std = np.std(opt_pars[:,3])
e_A_mean = np.mean(opt_pars[:,4])
e_A_std = np.std(opt_pars[:,4])
e_C_mean = np.mean(opt_pars[:,5])
e_C_std = np.std(opt_pars[:,5])
d_1_mean = np.mean(opt_pars[:,6])
d_1_std = np.std(opt_pars[:,6])
d_2_mean = np.mean(opt_pars[:,7])
d_2_std = np.std(opt_pars[:,7])
d_3_mean = np.mean(opt_pars[:,8])
d_3_std = np.std(opt_pars[:,8])
d_4_mean = np.mean(opt_pars[:,9])
d_4_std = np.std(opt_pars[:,9])
d_5_mean = np.mean(opt_pars[:,10])
d_5_std = np.std(opt_pars[:,10])
d_6_mean = np.mean(opt_pars[:,11])
d_6_std = np.std(opt_pars[:,11])
k_mean = np.mean(opt_pars[:,12])
k_std = np.std(opt_pars[:,12])
tau_1_mean = np.mean(opt_pars[:,13])
tau_1_std = np.std(opt_pars[:,13])
tau_2_mean = np.mean(opt_pars[:,14])
tau_2_std = np.std(opt_pars[:,14])
m_1_mean = np.mean(opt_pars[:,15])
m_1_std = np.std(opt_pars[:,15])
m_2_mean = np.mean(opt_pars[:,16])
m_2_std = np.std(opt_pars[:,16])
c_mean = np.mean(opt_pars[:,17])
c_std = np.std(opt_pars[:,17])
a_mean = np.mean(opt_pars[:,18])
a_std = np.std(opt_pars[:,18])
h_mean = np.mean(opt_pars[:,19])
h_std = np.std(opt_pars[:,19])
alpha_mean = np.mean(opt_pars[:,20])
alpha_std = np.std(opt_pars[:,20])
x_1_mean = np.mean(opt_pars[:,21])
x_1_std = np.std(opt_pars[:,21])
x_2_mean = np.mean(opt_pars[:,22])
x_2_std = np.std(opt_pars[:,22])
x_3_mean = np.mean(opt_pars[:,23])
x_3_std = np.std(opt_pars[:,23])
x_4_mean = np.mean(opt_pars[:,24])
x_4_std = np.std(opt_pars[:,24])
x_5_mean = np.mean(opt_pars[:,25])
x_5_std = np.std(opt_pars[:,25])
x_6_mean = np.mean(opt_pars[:,26])
x_6_std = np.std(opt_pars[:,26])
x_7_mean = np.mean(opt_pars[:,27])
x_7_std = np.std(opt_pars[:,27])
x_8_mean = np.mean(opt_pars[:,28])
x_8_std = np.std(opt_pars[:,28])
x_9_mean = np.mean(opt_pars[:,29])
x_9_std = np.std(opt_pars[:,29])
x_10_mean = np.mean(opt_pars[:,30])
x_10_std = np.std(opt_pars[:,30])
x_11_mean = np.mean(opt_pars[:,31])
x_11_std = np.std(opt_pars[:,31])
x_12_mean = np.mean(opt_pars[:,32])
x_12_std = np.std(opt_pars[:,32])

In [ ]:
# print a table of parameter means and standard deviations
print(tabulate([["e_P", "%f +- %f" % (e_P_mean, e_P_std)], ["e_T", "%f +- %f" % (e_T_mean, e_T_std)], ["e_S", "%f +- %f" % (e_S_mean, e_S_std)], ["e_A", "%f +- %f" % (e_A_mean, e_A_std)], ["e_C", "%f +- %f" % (e_C_mean, e_C_std)], ["d_1", "%f +- %f" % (d_1_mean, d_1_std)], ["d_2", "%f +- %f" % (d_2_mean, d_2_std)], ["d_3", "%f +- %f" % (d_3_mean, d_3_std)], ["d_4", "%f +- %f" % (d_4_mean, d_4_std)], ["d_5", "%f +- %f" % (d_5_mean, d_5_std)], ["d_6", "%f +- %f" % (d_6_mean, d_6_std)], ["k", "%f +- %f" % (k_mean, k_std)], ["tau_1", "%f +- %f" % (tau_1_mean, tau_1_std)], ["tau_2", "%f +- %f" % (tau_2_mean, tau_2_std)], ["m_1", "%f +- %f" % (m_1_mean, m_1_std)], ["m_2", "%f +- %f" % (m_2_mean, m_2_std)], ["c", "%f +- %f" % (c_mean, c_std)], ["a", "%f +- %f" % (a_mean, a_std)], ["h", "%f +- %f" % (h_mean, h_std)], ["alpha", "%f +- %f" % (alpha_mean, alpha_std)], ["x_1", "%f +- %f" % (x_1_mean, x_1_std)], ["x_2", "%f +- %f" % (x_2_mean, x_2_std)], ["x_3", "%f +- %f" % (x_3_mean, x_3_std)], ["x_4", "%f +- %f" % (x_4_mean, x_4_std)], ["x_5", "%f +- %f" % (x_5_mean, x_5_std)], ["x_6", "%f +- %f" % (x_6_mean, x_6_std)], ["x_7", "%f +- %f" % (x_7_mean, x_7_std)], ["x_8", "%f +- %f" % (x_8_mean, x_8_std)], ["x_9", "%f +- %f" % (x_9_mean, x_9_std)], ["x_10", "%f +- %f" % (x_10_mean, x_10_std)], ["x_11", "%f +- %f" % (x_11_mean, x_11_std)], ["x_12", "%f +- %f" % (x_12_mean, x_12_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))

In [ ]:
# save means and std devs to a file
np.savetxt(filename_root+'-param-means-stds.txt', [e_P_mean, e_P_std, e_T_mean, e_T_std, e_S_mean, e_S_std, e_A_mean, e_A_std, e_C_mean, e_C_std, d_1_mean, d_1_std, d_2_mean, d_2_std, d_3_mean, d_3_std, d_4_mean, d_4_std, d_5_mean, d_5_std, d_6_mean, d_6_std, k_mean, k_std, tau_1_mean, tau_1_std, tau_2_mean, tau_2_std, m_1_mean, m_1_std, m_2_mean, m_2_std, c_mean, c_std, a_mean, a_std, h_mean, h_std, alpha_mean, alpha_std, x_1_mean, x_1_std, x_2_mean, x_2_std, x_3_mean, x_3_std, x_4_mean, x_4_std, x_5_mean, x_5_std, x_6_mean, x_6_std, x_7_mean, x_7_std, x_8_mean, x_8_std, x_9_mean, x_9_std, x_10_mean, x_10_std, x_11_mean, x_11_std, x_12_mean, x_12_std])

[Back to Top](#top)

# Plots <a name="plots"></a>

In [ ]:
# Create an instance of the Visualizer class, which will start a series of prompts
grapher = Visualizer(globals())

In [ ]:
# The Visualizer method make_graphs can be called without arguments to create simple graphs without labels and with
#  default size and colors
grapher.make_graphs()

[Back to Top](#top)

# No Optimization Run <a name="no-opt"></a>

In [ ]:
%%time

# run the solver with authors' published parameters
data_no_opt = model(authors_params, y0)

[Back to Top](#top)

# Run Optimizations for Multiple Patients <a name="runMultiple" />

In [ ]:
# Enter the patient number to start and end the optimization loop
start_patient = 1
end_patient = 10

for patient in range(start_patient, end_patient+1):
    print(f"\033[1mCurrent Patient: #{patient}\033[0m")
    # Change the data set to cycle through patients in here. Will only work for Nelson TSST data, including subtype
    #  data sets (as the basal data sets are only mean concentrations, not individual patients)
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,patient+1], nelson.cortisol[:,0], nelson.cortisol[:,patient+1]]
    y0 = [0, 0, 0, data_to_match[1][0], data_to_match[3][0]]
    
    opt_pars_tmp, simData_all_tmp = optimize.run(cost_fun, model, data_to_match, y0, bounds, num_iter=5)
    
    if patient == start_patient:
        simData_all_multiple = simData_all_tmp
        opt_pars_multiple = opt_pars_tmp
    else:
        simData_all_multiple = np.hstack((simData_all_multiple, simData_all_tmp))
        opt_pars_multiple = np.hstack((opt_pars_multiple, opt_pars_tmp))
        

[Back to Top](#top)

# Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions

[Back to Top](#top)